In [1]:
import psycopg2

In [2]:
conn = psycopg2.connect(
                database='postgres_db',
                user='postgres_user',
                password='admin',
                host='client.tuple-mia.com',
                port='5432',
                sslmode='require'
            )

In [3]:
temp = conn.cursor()

In [4]:
import yaml
with open('/home/ubuntu/MappingBuffer_event.yml', 'r') as f:
    doc = yaml.load(f)

In [5]:
doc['table_map']

{'CUSTOMER_CONTACT': 'contact',
 'CUSTOMER_MASTER': 'users',
 'CUSTOMER_SECONDARY': 'temp_customer_secondary',
 'EVENT_LOG': 'event',
 'EVENT_MASTER': 'event_master',
 'TRANSACTION_MASTER': 'trans'}

In [6]:
doc['column_map']

{'CUSTOMER_CONTACT': {'cust_id': 'id',
  'email_id': 'email',
  'facebook_id': 'facebook_id',
  'firstname': 'first_name',
  'lastname': 'last_name',
  'phone_number': 'phone'},
 'CUSTOMER_MASTER': {'birthday': 'birthday',
  'create_date': 'create_date',
  'cust_id': 'id'},
 'CUSTOMER_SECONDARY': {'cust_id': 'cust_id'},
 'EVENT_LOG': {'action_type': 'actiontype',
  'cust_id': 'from_id',
  'product_id': 'to_id',
  'timestamp': 'timestamp'},
 'EVENT_MASTER': {'action_category_id': None,
  'action_category_name': None,
  'action_name': 'action_desc',
  'action_type': 'actiontype'},
 'TRANSACTION_MASTER': {'cust_id': 'userid',
  'product_id': 'product_id',
  'quantity': 'quantity',
  'renewal': 'renewal',
  'revenue': 'revenue',
  'timestamp': 'purchase_date'}}

In [7]:
query = "SELECT * FROM %s where %s = ( SELECT %s FROM %s WHERE %s = 'like')"%(
        doc['table_map']['EVENT_LOG'],
        doc['column_map']['EVENT_LOG']['action_type'],
        doc['column_map']['EVENT_MASTER']['action_type'],
        doc['table_map']['EVENT_MASTER'],
        doc['column_map']['EVENT_MASTER']['action_name']        
    )

In [8]:
print query

SELECT * FROM event where actiontype = ( SELECT actiontype FROM event_master WHERE action_desc = 'like')


In [9]:
temp.execute(query)

In [10]:
event = temp.fetchall()

In [ ]:
len(event)

In [11]:
colnames = [desc[0] for desc in temp.description]

In [ ]:
colnames

In [12]:
import pandas as pd

In [13]:
event_df = pd.DataFrame(event, columns=colnames)

In [ ]:
event_df.head()

In [14]:
event_df.rename(columns={doc['column_map']['EVENT_LOG']['cust_id']:'cust_id'}, inplace = True)
event_df.rename(columns={doc['column_map']['EVENT_LOG']['product_id']:'product_id'}, inplace = True)
event_df.rename(columns={doc['column_map']['EVENT_LOG']['action_type']:'actiontype'}, inplace = True)

In [15]:
event_data = event_df[['cust_id', 'product_id', 'actiontype']]

In [ ]:
event_data.head()

In [16]:
import findspark

In [17]:
findspark.init()

In [18]:
from pyspark import SparkContext
from pyspark.sql import SQLContext,Row
from pyspark.mllib.recommendation import ALS

In [19]:
sc = SparkContext(appName = "BuildProductRecommendations_event")

In [20]:
sqlContext = SQLContext(sc)

In [21]:
n_prod_views_rdd = sqlContext.createDataFrame(event_data).rdd

In [ ]:
n_prod_views_rdd.take(5)

In [22]:
training_rdd, test_rdd = n_prod_views_rdd.randomSplit([8,2], 1345)

In [ ]:
training_rdd.distinct().count()

In [ ]:
test_for_predict_rdd = test_rdd.map(lambda x : (x[0], x[1]))

In [ ]:
import math

In [ ]:
seed = 49247
iterations = 10
lambdas = [0.01, 0.1]
ranks = [16]
alphas = [1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 40.0, 80.0]
errors = [0 for x in range(len(alphas) * len(ranks) * len(lambdas))]
err_index = 0

In [ ]:
for lambda_ in  lambdas:
    for rank in ranks:
        for alpha in alphas:
            model = ALS.trainImplicit(training_rdd, rank, seed=seed, iterations=iterations,
                                           lambda_ = lambda_, alpha = alpha)
            predictions = model.predictAll(test_for_predict_rdd).map(lambda r : ((r[0], r[1]), r[2]))
            views_and_preds = test_rdd.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
            error = math.sqrt(views_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
            errors[err_index] = error
            err_index += 1
            print('For rank {0} at alpha: {1} and lambda: {2}, the RMSE is{3}'.format(rank, alpha, lambda_, error))

In [ ]:
model.recommendProducts(6253829,10)

In [23]:
model = ALS.trainImplicit(training_rdd, rank = 16, seed=49247, iterations=10, lambda_=0.1, alpha=80.0)

In [24]:
event_rec = model.recommendProductsForUsers(5).collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 466.0 failed 1 times, most recent failure: Lost task 2.0 in stage 466.0 (TID 775, localhost): java.lang.OutOfMemoryError: GC overhead limit exceeded
	at java.lang.Integer.valueOf(Integer.java:832)
	at scala.runtime.BoxesRunTime.boxToInteger(BoxesRunTime.java:65)
	at org.apache.spark.mllib.linalg.DenseMatrix.foreachActive(Matrices.scala:362)
	at org.apache.spark.mllib.recommendation.MatrixFactorizationModel$$anonfun$9.apply(MatrixFactorizationModel.scala:286)
	at org.apache.spark.mllib.recommendation.MatrixFactorizationModel$$anonfun$9.apply(MatrixFactorizationModel.scala:279)
	at scala.collection.Iterator$$anon$12.nextCur(Iterator.scala:434)
	at scala.collection.Iterator$$anon$12.hasNext(Iterator.scala:440)
	at org.apache.spark.util.collection.ExternalSorter.insertAll(ExternalSorter.scala:192)
	at or

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 52180)
----------------------------------------


Traceback (most recent call last):
  File "/home/ubuntu/anaconda2/lib/python2.7/SocketServer.py", line 295, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/home/ubuntu/anaconda2/lib/python2.7/SocketServer.py", line 321, in process_request
    self.finish_request(request, client_address)
  File "/home/ubuntu/anaconda2/lib/python2.7/SocketServer.py", line 334, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/home/ubuntu/anaconda2/lib/python2.7/SocketServer.py", line 655, in __init__
    self.handle()
  File "/opt/spark/python/pyspark/accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "/opt/spark/python/pyspark/serializers.py", line 545, in read_int
    raise EOFError
EOFError
